# รู้จักกับ Jupyter Notebook และโมดูล `sigproc.py`

เอกสารโน้ตบุ๊คเล่มนี้จัดทำขึ้นเพื่อสาธิตการใช้งานโมดูล `sigproc.py` ในการประมวลผลและวิเคราะห์สัญญาณ ซึ่งเป็นส่วนหนึ่งของการเรียนการสอนวิชา 01204325 การสื่อสารข้อมูลและเครือข่ายคอมพิวเตอร์ ไลบรารีหลักที่นำมาใช้งานในโมดูลมีดังนี้ (ทั้งหมดมีอยู่แล้วใน Anaconda)

* [NumPy](http://www.numpy.org) สำหรับการประมวลผลเชิงแมตริกซ์
* [SciPy](http://www.scipy.org) สำหรับการคำนวณ FFT รวมถึงการอ่านและเขียนไฟล์ .wav
* [Pandas](http://pandas.pydata.org) สำหรับการรวบรวมข้อมูลในรูปตาราง
* [Bokeh](http://bokeh.pydata.org/en/latest/index.html) ในการสร้างแผนภูมิและกราฟ

## วัตถุประสงค์
1. สร้างความคุ้นเคยกับอินเตอร์เฟสของ Jupyter (IPython) Notebook
1. เข้าใจคุณสมบัติของคลื่นที่อยู่ในรูปฟังก์ชันซายน์ ซึ่งประกอบไปด้วยแอพลิจูด ความถี่ และเฟส
1. เข้าใจการมองสัญญาณในเชิงเวลา (time domain) และเชิงความถี่ (frequency domain)

## การเตรียมตัว

อิมพอร์ตโมดูล `sigproc` จากนั้นเรียกใช้ฟังก์ชัน `start_notebook()` เพื่อเริ่มต้นใช้งานดังนี้

In [2]:
from sigproc import Signal,start_notebook
bkp = start_notebook()

Loading BokehJS ...

Notebook session ready.


## สร้างสัญญาณจากการผสมคลื่นซายน์

รูปแบบทั่วไปของคลื่นซายน์สามารถเขียนได้ตามฟังก์ชัน
$$s(t) = A\cdot\sin(2\pi ft + \phi)$$
โดยที่
* $A$ คือแอพลิจูด (amplitude)
* $f$ คือความถี่ (frequency) มีหน่วยเป็นเฮิร์ตซ์ (Hz)
* $\phi$ คือเฟสชิฟต์ (phase shift) มีหน่วยเป็นเรเดียน

โค้ดด้านล่างนิยามฟังก์ชันสำหรับสร้างคลื่นซายน์ 3 รูปแบบเพื่อนำไปสร้างเป็นสัญญาณด้วยโมดูล sigproc และพล็อตเป็นกราฟเชิงเวลา

In [3]:
from numpy import sin,pi

def sine1(t):
    return 1*sin(2*pi*1*t)

def sine2(t):
    return 1/3*sin(2*pi*3*t)

def sine3(t):
    return 1/5*sin(2*pi*5*t)

s1 = Signal(func=sine1)
s2 = Signal(func=sine2)
s3 = Signal(func=sine3)

s1.plot_time()
s2.plot_time()
s3.plot_time()

### การผสมสัญญาณ
โมดูล `sigproc` เตรียมฟังก์ชันและตัวดำเนินการที่หลากหลายเพื่อใช้ในการประมวลผลสัญญาณเบื้องต้น สัญญาณที่สร้างขึ้นมาทั้ง 3 สัญญาณสามารถนำมาผสมกันเป็นสัญญาณใหม่ได้โดยใช้ตัวดำเนินการ `+` ดังนี้

In [4]:
s = s1+s2+s3
s.plot_time(line_options=dict(color="orange",line_width=5))

เรียกใช้พร็อพเพอร์ตี `time_domain` เพื่อดึงข้อมูลของสัญญาณในรูปฟังก์ชันของเวลา ข้อมูลที่ส่งกลับมาจะอยู่ในรูปตารางที่ประกอบไปด้วยคอลัมน์ `t` แทนค่าเวลา และคอลัมน์ `a` แทนแอมพลิจูดของสัญญาณ ณ เวลาหนึ่ง ๆ

In [5]:
ts = s.time_domain
ts

t             a
0      0.000000  4.732924e-19
1      0.000045  8.548938e-04
2      0.000091  1.709787e-03
3      0.000136  2.564678e-03
4      0.000181  3.419567e-03
...         ...           ...
22045  0.999819 -3.419567e-03
22046  0.999864 -2.564678e-03
22047  0.999909 -1.709787e-03
22048  0.999955 -8.548938e-04
22049  1.000000  5.692076e-15

[22050 rows x 2 columns]

## การวิเคราะห์ความถี่ของสัญญาณ
ตามหลักการของ Fourier สัญญาณทุกสัญญาณสามารถตีความเป็นสัญญาณที่เกิดจากการนำเอาฟังก์ชันซายน์หลาย ๆ ฟังก์ชันมาผสมกันได้ ซึ่งเป็นการมองสัญญาณในเชิงความถี่ (frequency domain)

ใช้พร็อพเพอร์ตี `freq_domain` เพื่อตีความสัญญาณใน frequency domain โดยค่าที่ส่งคืนมาให้จะอยู่ในรูปตารางที่มี 3 คอลัมน์ คือ
* `f` แทนความถี่เป็นเฮิร์ตซ์
* `a` แทนแอพลิจูดสูงสุดของสัญญาณซายน์ในความถี่นั้น
* `p` แทนเฟสชิฟต์ของสัญญาณซายน์ในความถี่นั้น เทียบกับ_ฟังก์ชันโคซายน์_

In [6]:
fs = s.freq_domain
fs

f             a           p
0          0.0  1.015548e-16    0.000000
1          1.0  9.999924e-01  -89.991837
2          2.0  1.555847e-05   90.016327
3          3.0  3.333088e-01  -89.975510
4          4.0  3.572326e-05   90.032653
...        ...           ...         ...
11020  11020.0  3.877071e-08  179.959184
11021  11021.0  3.877071e-08  179.967347
11022  11022.0  3.877071e-08  179.975510
11023  11023.0  3.877070e-08  179.983673
11024  11024.0  3.877070e-08  179.991837

[11025 rows x 3 columns]

ทดลองนำเอาข้อมูลเชิงความถี่มาพล็อตเป็นกราฟแท่งโดยใช้เมท็อด `plot_freq()` เพื่อดูแอมพลิจูดของแต่ละความถี่ และ `plot_phase()` เพื่อดูเฟสชิฟท์ 

อย่างไรก็ตามจากตารางข้างต้นจะเห็นว่าข้อมูลประกอบไปด้วยความถี่ตั้งแต่ 0 Hz ถึง 11025 Hz และค่าแอมพลิจูดของความถี่ส่วนใหญ่มีค่าน้อยมาก ๆ เราจึงจะเลือกแสดงเฉพาะความถี่ที่มีค่ามากกว่า 0.01 ดังโค้ดด้านล่าง

In [7]:
s.plot_freq(min_amplitude=0.01,fig_options=dict(height=200))
s.plot_phase(min_amplitude=0.01,fig_options=dict(height=200))

## การนำเข้าข้อมูลสัญญาณจากไฟล์ .wav
คลาส `Signal` รองรับการนำเข้าข้อมูลสัญญาณจากไฟล์ .wav โดยระบุคีย์เวิร์ดอาร์กิวเมนต์ `wav_file` ดังตัวอย่าง

In [8]:
wav = Signal(wav_file='piano-c4.wav')

ซึ่งสามารถนำข้อมูลมาวิเคราะห์และแสดงผลได้เหมือนตัวอย่างที่ผ่านมา

In [9]:
figop = dict(y_range=None)  # use automatic y-range
wav.plot_time(fig_options=figop)
wav.plot_freq(fig_options=figop)
wav.plot_phase()

## ของแถม: เล่นเสียงจากข้อมูลสัญญาณ
คลาส `Signal` เตรียมเมท็อด `play()` สำหรับนำข้อมูลสัญญาณมาเล่นเป็นข้อมูลเสียง

In [10]:
wav.play()

ตัวอย่างด้านล่างแสดงการผสมคลื่นซายน์ 3 ความถี่เพื่อให้ได้เสียงเป็นคอร์ด C

In [11]:
# สร้างสัญญาณเปล่าที่มีความยาว 2 วินาที
chord_c = Signal(duration=2)

# กำหนดความถี่และแอมพลิจูดลงในสัญญาณโดยตรง
chord_c.set_freq(262,0.3,90) # โน้ต C
chord_c.set_freq(330,0.3,90) # โน้ต E
chord_c.set_freq(392,0.3,90) # โน้ต G

# แสดงกราฟในเชิงเวลาและความถี่
chord_c.plot_time()
chord_c.plot_freq()

# เล่นสัญญาณในรูปข้อมูลเสียง
chord_c.play()